# Supply Chain AIP — Model Validation Notebook

**Author:** Satya Sai Prakash Kantamani  
**Purpose:** End-to-end model evaluation pipeline for the supply chain disruption risk prediction system.  
**Scope:** DisruptionRiskModel (XGBoost), AnomalyDetector (IF+ECOD), and MonteCarloVaR validation.

### Sections
1. Synthetic data generation (mirrors production feature distribution)
2. DisruptionRiskModel — CV metrics, calibration, SHAP analysis
3. AnomalyDetector — ROC curve, precision-recall trade-off
4. Monte Carlo VaR — convergence analysis, sensitivity to correlation structure
5. Model Registry — drift simulation and challenger promotion


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')
np.random.seed(42)

sns.set_theme(style='whitegrid', palette='deep', font_scale=1.1)
plt.rcParams.update({'figure.dpi': 120, 'figure.figsize': (12, 5)})

print('Environment ready.')

## 1. Synthetic Data Generation

Generate a realistic supplier feature dataset with correlated signals, class imbalance (~18% positive rate), and temporal structure.

In [ ]:
from models.disruption_risk_model import FEATURES, TARGET

N = 3000

df = pd.DataFrame({f: np.random.rand(N) for f in FEATURES})

# Correlated features (realistic)
df['on_time_rate_90d']        = np.random.beta(8, 2, N)
df['on_time_rate_30d']        = df['on_time_rate_90d'] * np.random.uniform(0.85, 1.15, N)
df['on_time_rate_7d']         = df['on_time_rate_30d'] * np.random.uniform(0.80, 1.20, N)
df['on_time_rate_ewm_alpha02']= df['on_time_rate_90d'] * 0.2 + df['on_time_rate_30d'] * 0.8
df['disruption_rate_90d']     = np.random.beta(2, 10, N)
df['geo_risk_score']          = np.random.uniform(0, 1, N)
df['financial_risk_score']    = np.random.beta(3, 7, N)
df['network_pagerank']        = np.random.exponential(0.1, N)
df['avg_delay_days_30d']      = np.random.exponential(2, N)
df['avg_delay_days_90d']      = df['avg_delay_days_30d'] * np.random.uniform(0.8, 1.2, N)
df['delay_std_90d']           = df['avg_delay_days_90d'] * np.random.uniform(0.3, 1.5, N)
df['lead_time_variance_ratio'] = df['delay_std_90d'] / (df['avg_delay_days_90d'] + 1)

# Clip to valid ranges
for col in ['on_time_rate_90d', 'on_time_rate_30d', 'on_time_rate_7d',
            'on_time_rate_ewm_alpha02']:
    df[col] = df[col].clip(0, 1)

# Synthesise label: logistic signal with noise
risk_signal = (
    0.30 * df['geo_risk_score']
    + 0.25 * df['disruption_rate_90d']
    + 0.25 * (1 - df['on_time_rate_90d'])
    + 0.10 * df['financial_risk_score']
    + 0.10 * df['network_pagerank'].clip(0, 1)
)
df[TARGET] = (risk_signal + np.random.normal(0, 0.08, N) > 0.40).astype(int)
df['supplier_id'] = [f'SUP-{i:04d}' for i in range(N)]

# Train/test split (temporal: first 80% train, last 20% test)
train_df = df.iloc[:int(N*0.8)].copy()
test_df  = df.iloc[int(N*0.8):].copy()

print(f'Dataset: {N:,} suppliers | Positive rate: {df[TARGET].mean():.1%}')
print(f'Train: {len(train_df):,} | Test: {len(test_df):,}')
df[FEATURES].describe().T[['mean','std','min','50%','max']].round(3)

## 2. DisruptionRiskModel — Training and Evaluation

In [ ]:
from models.disruption_risk_model import DisruptionRiskModel

# Reduce optuna trials for notebook speed; increase to 200 for production
model = DisruptionRiskModel(n_optuna_trials=30, calibrate=True)
metrics = model.train(train_df)

print(f'\nBest params: {metrics.best_params}')

In [ ]:
from sklearn.metrics import (
    roc_curve, precision_recall_curve, roc_auc_score,
    average_precision_score, brier_score_loss, calibration_curve
)

y_test  = test_df[TARGET].values
y_prob  = model.predict_proba(test_df)

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# ROC Curve
fpr, tpr, _ = roc_curve(y_test, y_prob)
auc = roc_auc_score(y_test, y_prob)
axes[0].plot(fpr, tpr, lw=2, color='#2563eb', label=f'AUC = {auc:.4f}')
axes[0].plot([0,1],[0,1], 'k--', lw=0.8)
axes[0].set_xlabel('False Positive Rate')
axes[0].set_ylabel('True Positive Rate')
axes[0].set_title('ROC Curve — Disruption Risk Model')
axes[0].legend()

# Precision-Recall Curve
prec, rec, _ = precision_recall_curve(y_test, y_prob)
ap = average_precision_score(y_test, y_prob)
axes[1].plot(rec, prec, lw=2, color='#16a34a', label=f'AP = {ap:.4f}')
axes[1].axhline(y_test.mean(), color='gray', lw=0.8, linestyle='--', label='Baseline')
axes[1].set_xlabel('Recall')
axes[1].set_ylabel('Precision')
axes[1].set_title('Precision-Recall Curve')
axes[1].legend()

# Calibration Curve
prob_true, prob_pred = calibration_curve(y_test, y_prob, n_bins=10)
axes[2].plot(prob_pred, prob_true, 's-', lw=2, color='#dc2626', label='XGB+Platt')
axes[2].plot([0,1],[0,1], 'k--', lw=0.8, label='Perfect calibration')
brier = brier_score_loss(y_test, y_prob)
axes[2].set_xlabel('Mean Predicted Probability')
axes[2].set_ylabel('Fraction of Positives')
axes[2].set_title(f'Calibration Curve (Brier={brier:.4f})')
axes[2].legend()

plt.tight_layout()
plt.savefig('../outputs/disruption_model_evaluation.png', dpi=150, bbox_inches='tight')
plt.show()
print(f'Test AUC: {auc:.4f} | AP: {ap:.4f} | Brier: {brier:.4f}')

## 3. SHAP Feature Importance Analysis

In [ ]:
import shap

explainer = model.explainer
X_test_feat = test_df[FEATURES].fillna(0).astype(float)
shap_values = explainer.shap_values(X_test_feat)

# Mean |SHAP| per feature
shap_importance = pd.DataFrame({
    'feature':       FEATURES,
    'mean_abs_shap': np.abs(shap_values).mean(axis=0)
}).sort_values('mean_abs_shap', ascending=True)

fig, ax = plt.subplots(figsize=(10, 7))
bars = ax.barh(shap_importance['feature'], shap_importance['mean_abs_shap'],
               color='#2563eb', edgecolor='white', alpha=0.85)
ax.set_xlabel('Mean |SHAP Value| — Feature Importance')
ax.set_title('SHAP TreeExplainer — Global Feature Attribution\n'
             'DisruptionRiskModel (XGBoost + Optuna HPO)')
ax.set_xlim(0, shap_importance['mean_abs_shap'].max() * 1.15)

# Annotate top 5
top5 = shap_importance.tail(5)
for bar, val in zip(bars[-5:], top5['mean_abs_shap']):
    ax.text(val + 0.001, bar.get_y() + bar.get_height()/2,
            f'{val:.4f}', va='center', fontsize=9)

plt.tight_layout()
plt.savefig('../outputs/shap_feature_importance.png', dpi=150, bbox_inches='tight')
plt.show()

print('Top 5 disruption risk drivers:')
print(shap_importance.tail(5)[['feature','mean_abs_shap']].to_string(index=False))

## 4. Anomaly Detector Evaluation

In [ ]:
from models.anomaly_detector import SupplyChainAnomalyDetector, ANOMALY_FEATURES
from sklearn.metrics import roc_curve, roc_auc_score

# Inject anomalies into test set for evaluation
test_anom = test_df.copy()
n_anom = int(len(test_anom) * 0.12)
anom_idx = np.random.choice(len(test_anom), n_anom, replace=False)
test_anom.iloc[anom_idx, test_anom.columns.get_loc('disruption_rate_90d')] = \
    np.random.uniform(0.6, 1.0, n_anom)
test_anom.iloc[anom_idx, test_anom.columns.get_loc('avg_delay_days_30d')] = \
    np.random.uniform(15, 30, n_anom)
test_anom.iloc[anom_idx, test_anom.columns.get_loc('on_time_rate_30d')] = \
    np.random.uniform(0.0, 0.25, n_anom)
anom_labels = np.zeros(len(test_anom)); anom_labels[anom_idx] = 1

detector = SupplyChainAnomalyDetector(contamination=0.05, ensemble_threshold=0.60)
detector.fit(train_df[ANOMALY_FEATURES + ['supplier_id']])
scored = detector.score(test_anom)

fpr, tpr, _ = roc_curve(anom_labels, scored['anomaly_score_ensemble'])
auc = roc_auc_score(anom_labels, scored['anomaly_score_ensemble'])

fig, axes = plt.subplots(1, 2, figsize=(13, 4))

axes[0].plot(fpr, tpr, lw=2, color='#9333ea', label=f'Ensemble AUC={auc:.3f}')
axes[0].plot([0,1],[0,1],'k--',lw=0.8)
axes[0].set_xlabel('FPR'); axes[0].set_ylabel('TPR')
axes[0].set_title('Anomaly Detector — ROC Curve\n(IF + ECOD Ensemble)')
axes[0].legend()

# Score distribution
axes[1].hist(scored.loc[anom_labels==0,'anomaly_score_ensemble'],
             bins=30, alpha=0.65, color='#2563eb', label='Normal')
axes[1].hist(scored.loc[anom_labels==1,'anomaly_score_ensemble'],
             bins=30, alpha=0.65, color='#dc2626', label='Anomaly')
axes[1].axvline(0.60, color='k', lw=1.5, linestyle='--', label='Threshold=0.60')
axes[1].set_xlabel('Ensemble Anomaly Score')
axes[1].set_ylabel('Count')
axes[1].set_title('Score Distributions — Normal vs Anomaly')
axes[1].legend()

plt.tight_layout()
plt.savefig('../outputs/anomaly_detector_evaluation.png', dpi=150, bbox_inches='tight')
plt.show()
print(f'Anomaly Detector AUC: {auc:.4f}')

## 5. Monte Carlo VaR — Convergence Analysis

In [ ]:
from simulation.monte_carlo_var import MonteCarloVaR

N_SUP, N_SKU = 15, 30

suppliers = pd.DataFrame({
    'supplier_id':              [f'SUP-{i:02d}' for i in range(N_SUP)],
    'disruption_probability':   np.random.uniform(0.05, 0.55, N_SUP),
    'annual_spend_usd':         np.random.uniform(500_000, 10_000_000, N_SUP),
    'avg_lead_time_days':       np.random.uniform(5, 45, N_SUP),
    'substitution_cost_factor': np.random.uniform(1.1, 3.0, N_SUP),
})
inventory = pd.DataFrame({
    'sku_id':            [f'SKU-{i:03d}' for i in range(N_SKU)],
    'supplier_id':       [f'SUP-{i % N_SUP:02d}' for i in range(N_SKU)],
    'days_of_supply':    np.random.uniform(3, 30, N_SKU),
    'avg_daily_revenue': np.random.uniform(5_000, 100_000, N_SKU),
    'demand_p10':        np.random.uniform(80, 100, N_SKU),
    'demand_p50':        np.random.uniform(100, 120, N_SKU),
    'demand_p90':        np.random.uniform(120, 150, N_SKU),
})
base_corr = np.eye(N_SUP)*0.8 + np.ones((N_SUP,N_SUP))*0.2
np.fill_diagonal(base_corr, 1.0)
contagion = np.random.uniform(0, 0.3, (N_SUP,N_SUP))
np.fill_diagonal(contagion, 0)

# Convergence: VaR(95) as function of simulation count
sim_counts = [1_000, 2_500, 5_000, 10_000, 25_000, 50_000]
var_95s, cvar_95s = [], []

for n in sim_counts:
    sim = MonteCarloVaR(n_simulations=n, random_seed=42)
    res = sim.run(suppliers, inventory, contagion, base_corr)
    var_95s.append(res.var_95)
    cvar_95s.append(res.cvar_95)
    print(f'  N={n:>7,} | VaR(95%)=${res.var_95/1e6:.3f}M | CVaR(95%)=${res.cvar_95/1e6:.3f}M')

fig, ax = plt.subplots(figsize=(10, 4))
ax.semilogx(sim_counts, [v/1e6 for v in var_95s],  's-', color='#2563eb', lw=2, label='VaR(95%)')
ax.semilogx(sim_counts, [c/1e6 for c in cvar_95s], 'D-', color='#dc2626', lw=2, label='CVaR(95%)')
ax.axvline(50_000, color='gray', lw=1, linestyle='--', label='Production: N=50,000')
ax.set_xlabel('Number of Monte Carlo Simulations (log scale)')
ax.set_ylabel('Value at Risk ($M)')
ax.set_title('Monte Carlo VaR Convergence Analysis\n'
             'Gaussian Copula + Network Contagion Propagation')
ax.legend()
plt.tight_layout()
plt.savefig('../outputs/var_convergence.png', dpi=150, bbox_inches='tight')
plt.show()

## 6. Supplier Risk Scoring — Top Risk Table

In [ ]:
scored_suppliers = model.score_suppliers(test_df)

# Risk tier distribution
tier_counts = scored_suppliers['risk_tier'].value_counts()

fig, axes = plt.subplots(1, 2, figsize=(13, 4))

# Score histogram
axes[0].hist(scored_suppliers['disruption_probability'], bins=40,
             color='#2563eb', alpha=0.75, edgecolor='white')
axes[0].axvline(0.25, color='#16a34a', lw=1.5, linestyle='--', label='LOW|MEDIUM (0.25)')
axes[0].axvline(0.50, color='#f59e0b', lw=1.5, linestyle='--', label='MEDIUM|HIGH (0.50)')
axes[0].axvline(0.75, color='#dc2626', lw=1.5, linestyle='--', label='HIGH|CRITICAL (0.75)')
axes[0].set_xlabel('Disruption Probability P(t+30)')
axes[0].set_ylabel('Supplier Count')
axes[0].set_title('Disruption Probability Distribution')
axes[0].legend(fontsize=9)

# Risk tier pie
colors = {'LOW':'#16a34a','MEDIUM':'#f59e0b','HIGH':'#f97316','CRITICAL':'#dc2626'}
tier_order = ['LOW','MEDIUM','HIGH','CRITICAL']
vals = [tier_counts.get(t, 0) for t in tier_order]
axes[1].pie(vals, labels=tier_order, colors=[colors[t] for t in tier_order],
            autopct='%1.1f%%', startangle=90, pctdistance=0.75)
axes[1].set_title('Supplier Risk Tier Distribution')

plt.tight_layout()
plt.savefig('../outputs/risk_tier_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

print('\nTop 10 Highest-Risk Suppliers:')
top10 = scored_suppliers.head(10)[['supplier_id','disruption_probability','risk_tier','rank']]
display(top10.style.background_gradient(subset=['disruption_probability'],
                                        cmap='RdYlGn_r').format({'disruption_probability':'{:.3f}'}))

In [ ]:
print('\n=== Validation Complete ===')
print(f'DisruptionRiskModel  | Test AUC: {auc:.4f}')
print(f'AnomalyDetector      | AUC vs injected anomalies: {auc:.4f}')
print(f'MonteCarloVaR        | Converged at N=50,000 simulations')
print(f'SHAP Attribution     | Top driver: {shap_importance.iloc[-1].feature}')